# 🧪 Fine-Tuning with Unsloth: Guided Notebook
This notebook walks you through fine-tuning a small LLM using Unsloth.
We will:
- Load a JSONL dataset
- Initialize a PEFT-compatible model
- Configure training
- Save and reload the model
- Generate outputs to verify results

## 🔍 Step 1: Environment Check

In [ ]:
!pip install -q transformers accelerate datasets unsloth peft

## 📁 Step 2: Load Your Dataset
Ensure your file follows Alpaca-style JSONL with `instruction` and `output`.

In [ ]:
from datasets import load_dataset
raw = load_dataset('json', data_files='data/custom_training.jsonl')['train']
raw = raw.map(lambda ex: {'prompt': ex['instruction'], 'completion': ex['output']})
print(raw[0])

## 🧠 Step 3: Initialize the Model with Unsloth

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/zephyr-1.3b-bnb-4bit',
    max_seq_length=2048,
    load_in_4bit=True
)

## ⚙️ Step 4: Configure Training Parameters

In [ ]:
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir='./checkpoints/my_model',
    per_device_train_batch_size=2,
    num_train_epochs=1,
    fp16=True,
    logging_steps=10,
    save_strategy='epoch'
)

## 🚀 Step 5: Train the Model

In [ ]:
FastLanguageModel.for_inference(model)
model.train_model(dataset=raw, args=args)

## 💾 Step 6: Save and Reload Model

In [ ]:
model.save_pretrained('./checkpoints/my_model')
# To reload later:
# model = model.from_pretrained('./checkpoints/my_model')

## 🧪 Step 7: Generate a Sample Output

In [ ]:
prompt = "Explain coroutines in Python"
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(model.device)
out = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.decode(out[0], skip_special_tokens=True))